In [159]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score  
from tqdm import tqdm 


### Model Construction

In [160]:
class ProductLayer(nn.Module):
    def __init__(self, num_fields, embed_dim, l1_hidden_dim,mode='inner'):
        super().__init__()
        self.mode = mode
        self.num_fields = num_fields
        self.bias = torch.nn.Parameter(torch.zeros(1))
        
        if mode == 'outer':
            # 外积模式需要参数矩阵降维
            self.weight = nn.Parameter(torch.randn(embed_dim, embed_dim))
        
        # 线性部分权重 (保留原始Embedding)
        self.linear_weights = nn.Linear(num_fields * embed_dim, num_fields * embed_dim)

        self.lz_fc = nn.Linear(num_fields * embed_dim, l1_hidden_dim)
        self.lp_fc = nn.Linear(num_fields * (num_fields-1) // 2, l1_hidden_dim)
        
    def forward(self, embeddings):
        """
        Input: 
            embeddings: List of tensors [B, embed_dim] * num_fields
        Output:
            product_output: Tensor [B, output_dim]
        """
        batch_size = embeddings[0].size(0)
        stacked = torch.stack(embeddings, dim=1)  # [B, num_fields, embed_dim]
        
        # 线性部分
        linear_part = stacked.view(batch_size, -1)  # [B, num_fields*embed_dim]
        linear_out = self.linear_weights(linear_part)
        
        # 交互部分
        if self.mode == 'inner':
            # 内积模式：所有向量两两点积
            inner_products = []
            for i in range(self.num_fields):
                for j in range(i+1, self.num_fields):
                    ip = torch.sum(embeddings[i] * embeddings[j], dim=1, keepdim=True)
                    inner_products.append(ip)
            product_out = torch.cat(inner_products, dim=1)  # [B, C(num_fields,2)]
            
        elif self.mode == 'outer':
            # 外积模式：外积矩阵压缩
            outer_products = []
            for i in range(self.num_fields):
                for j in range(i+1, self.num_fields):
                    op = torch.bmm(embeddings[i].unsqueeze(2),  # [B, embed_dim, 1]
                                 embeddings[j].unsqueeze(1))    # [B, 1, embed_dim]
                    
                    #op:# [B, embed_dim, embed_dim], self.weight # [embed_dim, embed_dim]
                    op = (op * self.weight).sum(dim = (1,2), keepdim = True).squeeze(1)  # 降维,将对位运算的结果相加,并且保持维度为[B,1]
                    outer_products.append(op)
            product_out = torch.cat(outer_products, dim=1)
        
        # 合并线性与交互部分
        lz_out = self.lz_fc(linear_out)
        lp_out = self.lp_fc(product_out)
        combined = lz_out + lp_out
        combined += self.bias
        return torch.relu(combined)

In [161]:
class PNN(nn.Module):
    def __init__(self,fields_feature, embed_dim, hidden_dims, mode = 'inner'):
        """
        fields_feature: list, 每个特征域的类别数目
        embed_dim: 嵌入的维度
        hidden_dims: list, 隐藏层维度
        mode: 计算方式
        """
        super().__init__()
        #Embedding层
        self.embeddings = nn.ModuleList([
            nn.Embedding(input_dim, embed_dim) for input_dim in fields_feature
        ])

        # 乘积层
        self.product_layers = ProductLayer(len(fields_feature), embed_dim, hidden_dims[0],mode)

        # 全连接层
        product_dim = hidden_dims[0]

        fc_layer = []
        for hidden_dim in hidden_dims:
            fc_layer.append(nn.Linear(product_dim, hidden_dim))
            fc_layer.append(nn.ReLU())
            product_dim = hidden_dim
        self.fc = nn.Sequential(*fc_layer)

        # 输出层
        self.output_layer = nn.Linear(hidden_dims[-1], 1)

    def forward(self,field):
        embeds = [self.embeddings[i](field[:, i]) for i in range(len(field[0]))]
        product_output = self.product_layers(embeds)
        fc_output = self.fc(product_output)
        output = self.output_layer(fc_output)
        return torch.sigmoid(output)

        

### Data Construction

In [162]:
# 生成训练数据
class CTRDataset(Dataset):
    def __init__(self, num_samples, fields_feature):
        self.num_fields = len(fields_feature)
        self.data = []
        self.labels = []
        
        # 随机生成样本数据
        for _ in range(num_samples):
            features = [
                torch.randint(0, feat_size, (1,)).item()
                for feat_size in fields_feature
            ]
            self.data.append(features)
            # 随机生成伪标签（实际使用时替换为真实标签）
            label = 1 if features[0] > fields_feature[0]//2 else 0
            self.labels.append(label)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return (
            torch.tensor(self.data[idx], dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.float32)
        )

In [ ]:
# 配置训练参数
num_samples = 1000  # 样本数量
num_fields = 5      # 特征域数量
embed_dim = 16
fields_feature = [15, 20, 10, 12, 18]  # 每个特征域的类别数目
num_epochs = 10
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 创建数据集和数据加载器
dataset = CTRDataset(num_samples, fields_feature)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 初始化模型实例
model = PNN(
    fields_feature=fields_feature,
    embed_dim=embed_dim,
    hidden_dims=[64, 32],
    mode='inner'
).to(device)

# 定义损失函数和优化器
criterion = nn.BCELoss()  # 二分类任务
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


print(model)

PNN(
  (embeddings): ModuleList(
    (0): Embedding(15, 16)
    (1): Embedding(20, 16)
    (2): Embedding(10, 16)
    (3): Embedding(12, 16)
    (4): Embedding(18, 16)
  )
  (product_layers): ProductLayer(
    (linear_weights): Linear(in_features=80, out_features=80, bias=True)
    (lz_fc): Linear(in_features=80, out_features=64, bias=True)
    (lp_fc): Linear(in_features=10, out_features=64, bias=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)


### Model Training

In [164]:
pbar = tqdm(range(num_epochs))
for epoch in pbar:
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    
    for batch_idx, (features, labels) in enumerate(dataloader):
        features = features.to(device)
        labels = labels.to(device).view(-1, 1)  # 调整标签维度
        
        # 前向传播
        outputs = model(features)
        loss = criterion(outputs, labels)
        
        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 记录统计信息
        total_loss += loss.item()
        preds = (outputs > 0.95).float()  # 预测结果阈值为0.95
        all_preds.extend(preds.cpu().detach().numpy())
        all_labels.extend(labels.cpu().detach().numpy())
    
    # 计算epoch指标
    epoch_loss = total_loss / len(dataloader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    
    # 打印训练进度
    pbar.set_postfix(loss=epoch_loss, acc=epoch_acc)
    pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}]")

Epoch [10/10]: 100%|██████████| 10/10 [00:00<00:00, 22.26it/s, acc=1, loss=0.000227]
